In [ ]:
import os
import tarfile
import shutil
import dask
import requests
import time
import psutil

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from astrohack import holog
from astrohack.extract_holog import extract_holog
from astrohack.dio import _load_holog_file
from astrohack._utils._io import _make_ant_pnt_dict, _load_pnt_dict
%matplotlib widget
from dask.distributed import Client, LocalCluster
import matplotlib.pyplot as plt
from scipy import spatial

In [ ]:
# Only a single holography of VLA beam over scans 2,4,6,8,10,12,14,16,18,20,22 (ALMA can map an entire beam in a single scan).
#holog_obs_description = {0 :{'scans':[2,4,6,8,10,12,14,16,18,20,22],'ant':{'EA25':['EA08', 'EA04', 'EA12', 'EA27', 'EA19', 'EA07', 'EA21', 'EA03', 'EA11', 'EA05', 'EA01', 'EA09', 'EA24', 'EA22', 'EA14', 'EA18', 'EA02']}}}
holog_obs_description = {0 :{'scans':[2,4,6,8,10,12,14,16,18,20,22],'ant':{'EA25':['EA04']}}}
holog_obs_description['ddi'] = np.arange(16)

sel_ddi = [1]

extract_holog(
    ms_name='ea25/ea25_cal_small.ms', 
    holog_name='ea25/ea25_cal_small_1ref', 
    holog_obs_dict=holog_obs_description,
    data_col='DATA',
    subscan_intent='MIXED',
    parallel=False,
    overwrite=True,
    sel_ddi=sel_ddi
)

In [1]:
from astrohack import holog
grid_size = None
cell_size = None
grid_interpolation_mode = 'linear' #'nearest' 'linear' 'cubic'
chan_average = True
chan_tolerance_factor = 0.005
reference_scaling_frequency = None #1.5*10**10 #None #In Hz
#spw_average if True users has to give reference_scaling_frequency
scan_average = True
holog(holog_file='ea25/ea25_cal_small_1ref.holog.zarr', padding_factor=50, 
      parallel=False,
      grid_size=grid_size, cell_size=cell_size, grid_interpolation_mode=grid_interpolation_mode,
      chan_average = chan_average, chan_tolerance_factor = chan_tolerance_factor,
      reference_scaling_frequency=reference_scaling_frequency,
      scan_average = scan_average,
      to_stokes=True,
     )

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


[2023-03-07 01:47:54.964968 System Info]: Loading holography file ea25/ea25_cal_small_1ref.holog.zarr ...
[2023-03-07 01:47:54.968592 System Info]: Cell size: [0.00062476 0.00062476] Grid size [31 31]
(-0.024954214575700462-0.1105716438323725j) (-0.10330307074764278+0.016614910158750714j) (-0.024954214575700462-0.1105716438323725j) (-0.10330307074764278+0.016614910158750714j)
0.11335255288207294 0.10463068223745771 0.11335255288207294 0.10463068223745771
-102.7176620337067 170.86299444950316
[2023-03-07 01:47:55.153460 System Info]: [_calculate_aperture_pattern] Calculating aperture illumination pattern ...
[2023-03-07 01:47:55.267432 System Info]: [_holog_chunk] Applying phase correction ...


In [ ]:
%%time
from astrohack.panel import panel
image = 'ea25/ea25_cal_small_1ref'
#ddi = [1] 
ddi = None

#["mean", "rigid", "corotated_scipy", "corotated_lst_sq", "corotated_robust", "xy_paraboloid",
#              "rotated_paraboloid", "full_paraboloid_lst_sq"]

panel_kind = 'rigid'

panel(
    holog_image=image, 
    panel_kind=panel_kind,
    outfile='ea25/ea25_cal_small_1ref'+panel_kind,
    aipsdata=False, 
    unit='miliinches', 
    save_mask=True,
    save_phase=True,
    parallel=False
)

Replace data for Panel

In [ ]:
from astrohack._utils._fits_io import _read_fits, _aips_holog_to_xds

In [ ]:
xds = _aips_holog_to_xds('panel-input-data/amp.fits','panel-input-data/dev.fits')

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(xds.DEVIATION)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.imshow(xds.AMPLITUDE)
plt.show()

In [ ]:
import xarray as xr

In [ ]:
xds.to_zarr('ea25_aips.image.zarr')

In [ ]:
from astrohack.panel import panel
image = '/Users/jsteeb/Dropbox/astrohack/examples/ea25/ea25_aips.image.zarr/24/1'
#ddi = [1] 
ddi = None

panel_kind = 'rigid'

panel(
    holog_image=image, 
    outfile='ea25/ea25_aips_'+panel_kind,
    panel_kind = panel_kind,
    aipsdata=True, 
    unit='miliinches', #mm, miliinches
    save_mask=True,
    save_phase=True,
    parallel=False,
    telescope="VLA",
    basename="EA"
)